In [1]:
# ============================================================
# 🧠 Dual Neural Expert v3.0 — InfoNCE + True ST Embedding
#  - Mục tiêu:
#    • Anchor@1 cao hơn nhờ in-batch hard negatives (InfoNCE)
#    • Context@1 ổn định, không lệch so với encoder gốc
#    • Không đụng internal .auto_model, dùng SentenceTransformer đúng chuẩn
# ============================================================

import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer

# ------------------------------------------------------------
# ⚙️ CONFIGURATION
# ------------------------------------------------------------
DATA_FILE = "Advice.csv"                      # synthetic Advice dataset
MODEL_NAME = "google/embeddinggemma-300m"     # base encoder
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

EPOCHS = 20
BATCH_SIZE = 32
LR = 2e-5
SEED = 42

# InfoNCE temperature
TEMP_ANCHOR = 0.07
TEMP_CONTEXT = 0.07

# Loss weights
LAMBDA_ANCHOR  = 1.0
LAMBDA_CONTEXT = 2.0   # boost context
LAMBDA_HASCTX  = 0.5

SAVE_PATH = "models_anchor/dual_expert_Advice_v3_0"
os.makedirs(SAVE_PATH, exist_ok=True)

random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)


# ------------------------------------------------------------
# 🧩 Dataset
#   Expect columns: question, expert_label, anchor, has_context, context_name
# ------------------------------------------------------------
class DualAnchorDataset(Dataset):
    def __init__(self, df):
        self.data = df.to_dict("records")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data[idx]

        q = str(row["question"])
        anchor = str(row["anchor"])

        has_context = int(row.get("has_context", 0))

        if has_context == 1 and pd.notna(row.get("context_name", "")):
            context = str(row["context_name"])
        else:
            context = ""

        return q, anchor, context, has_context


# ------------------------------------------------------------
# 🧠 Dual Expert Adapter
#   - Nhận embedding từ SentenceTransformer
#   - Trả ra anchor_emb, context_emb, has_context_logit
# ------------------------------------------------------------
class DualExpertAdapter(nn.Module):
    def __init__(self, input_dim=768, hidden_dim=256):
        super().__init__()

        self.anchor_adapter = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )

        self.context_adapter = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )

        self.has_context_head = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x):
        # x: (B, D) from SentenceTransformer
        anchor_emb = F.normalize(self.anchor_adapter(x), p=2, dim=1)
        context_emb = F.normalize(self.context_adapter(x), p=2, dim=1)
        has_ctx_logit = self.has_context_head(x).squeeze(-1)
        return anchor_emb, context_emb, has_ctx_logit


# ------------------------------------------------------------
# 🔧 Helper: encode text with SentenceTransformer
#   - Đảm bảo dùng pooling chuẩn của SentenceTransformer
# ------------------------------------------------------------
def encode_texts(encoder: SentenceTransformer, texts, batch_size=64):
    # texts: list[str]
    # Trả về: Tensor (N, D) trên DEVICE
    with torch.no_grad():
        emb = encoder.encode(
            texts,
            batch_size=batch_size,
            convert_to_tensor=True,
            device=DEVICE,
            show_progress_bar=False
        )
    return emb


# ------------------------------------------------------------
# 🚀 Training loop — Dual Expert v3.0
# ------------------------------------------------------------
def train_dual_expert_v3():
    print("\n=== 🧩 Training Dual Expert v3.0 — InfoNCE Edition ===")

    # -----------------------------
    # Load & clean data
    # -----------------------------
    df = pd.read_csv(DATA_FILE)
    df = df[df["expert_label"] == "Advice"].dropna(subset=["question", "anchor"])

    if "context_name" not in df.columns:
        df["context_name"] = ""
    if "has_context" not in df.columns:
        df["has_context"] = 0

    df["has_context"] = df["has_context"].fillna(0).astype(int)

    # Đảm bảo: nếu has_context = 0 thì context_name = ""
    df["context_name"] = df.apply(
        lambda x: str(x["context_name"]) if x["has_context"] == 1 and pd.notna(x["context_name"]) else "",
        axis=1
    )

    print(f"✅ Loaded {len(df)} Advice samples")

    dataset = DualAnchorDataset(df)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

    # -----------------------------
    # Base encoder (frozen)
    # -----------------------------
    base_encoder = SentenceTransformer(MODEL_NAME)
    base_encoder.to(DEVICE)
    base_encoder.eval()
    for p in base_encoder.parameters():
        p.requires_grad = False

    # Lấy dim từ SentenceTransformer (an toàn)
    try:
        input_dim = base_encoder.get_sentence_embedding_dimension()
    except Exception:
        # fallback nếu model không có hàm này
        test_emb = encode_texts(base_encoder, ["test"])
        input_dim = test_emb.size(1)

    adapter = DualExpertAdapter(input_dim=input_dim, hidden_dim=256).to(DEVICE)

    optimizer = torch.optim.AdamW(adapter.parameters(), lr=LR)
    bce_loss = nn.BCEWithLogitsLoss()

    use_amp = DEVICE == "cuda"
    scaler = torch.amp.GradScaler("cuda") if use_amp else None

    # -----------------------------
    # Training epochs
    # -----------------------------
    for epoch in range(EPOCHS):
        adapter.train()
        total_loss = 0.0
        total_anchor_loss = 0.0
        total_ctx_loss = 0.0
        total_hasctx_loss = 0.0

        pbar = tqdm(loader, desc=f"Epoch {epoch+1}/{EPOCHS}")

        for batch in pbar:
            q_texts, a_texts, c_texts, has_ctx = batch
            has_ctx = torch.tensor(has_ctx, dtype=torch.float32, device=DEVICE)

            if use_amp:
                ctx_mgr = torch.amp.autocast("cuda")
            else:
                # no-op context manager
                from contextlib import nullcontext
                ctx_mgr = nullcontext()

            with ctx_mgr:
                # 1) Encode texts -> embeddings (SentenceTransformer pooling chuẩn)
                q_emb = encode_texts(base_encoder, list(q_texts))
                a_emb = encode_texts(base_encoder, list(a_texts))

                # Context embeddings: encode cả batch (kể cả chỗ rỗng, sẽ mask sau)
                c_emb = encode_texts(base_encoder, list(c_texts))

                # 2) Adapter forward
                q_anchor, q_ctx, has_ctx_logit = adapter(q_emb)
                a_anchor, _, _ = adapter(a_emb)
                _, c_ctx, _ = adapter(c_emb)

                # ========== Anchor InfoNCE Loss ==========
                # sim(q_i, a_j) cho toàn batch
                sim_anchor = torch.matmul(q_anchor, a_anchor.T) / TEMP_ANCHOR   # (B, B)
                target = torch.arange(sim_anchor.size(0), device=DEVICE)       # mỗi q khớp anchor cùng index
                loss_anchor = F.cross_entropy(sim_anchor, target)

                # ========== Context InfoNCE Loss ==========
                mask_ctx = has_ctx > 0.5
                if mask_ctx.sum() > 1:
                    q_ctx_pos = q_ctx[mask_ctx]
                    c_ctx_pos = c_ctx[mask_ctx]

                    # in-batch similarity
                    sim_ctx = torch.matmul(q_ctx_pos, c_ctx_pos.T) / TEMP_CONTEXT
                    target_ctx = torch.arange(sim_ctx.size(0), device=DEVICE)
                    loss_ctx = F.cross_entropy(sim_ctx, target_ctx)
                else:
                    loss_ctx = torch.tensor(0.0, device=DEVICE)

                # ========== HasContext BCE ==========
                loss_hasctx = bce_loss(has_ctx_logit, has_ctx)

                # ========== Total loss ==========
                loss = (
                    LAMBDA_ANCHOR  * loss_anchor +
                    LAMBDA_CONTEXT * loss_ctx +
                    LAMBDA_HASCTX  * loss_hasctx
                )

            optimizer.zero_grad()
            if use_amp:
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                loss.backward()
                optimizer.step()

            total_loss += loss.item()
            total_anchor_loss += loss_anchor.item()
            total_ctx_loss += loss_ctx.item()
            total_hasctx_loss += loss_hasctx.item()

            step = pbar.n + 1
            pbar.set_postfix(
                loss=f"{total_loss/step:.4f}",
                anchor=f"{total_anchor_loss/step:.4f}",
                ctx=f"{total_ctx_loss/step:.4f}",
                hasctx=f"{total_hasctx_loss/step:.4f}"
            )

        n_steps = len(loader)
        print(
            f"✅ Epoch {epoch+1} | "
            f"Loss={total_loss/n_steps:.4f} | "
            f"Anchor={total_anchor_loss/n_steps:.4f} | "
            f"Context={total_ctx_loss/n_steps:.4f} | "
            f"HasCtx={total_hasctx_loss/n_steps:.4f}"
        )

    # -----------------------------
    # Save adapter + encoder
    # -----------------------------
    torch.save(adapter.state_dict(), os.path.join(SAVE_PATH, "dual_adapter_v3.pt"))
    base_encoder.save(SAVE_PATH)

    print(f"\n💾 Saved model & adapter to: {SAVE_PATH}")


# ------------------------------------------------------------
if __name__ == "__main__":
    train_dual_expert_v3()


/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



=== 🧩 Training Dual Expert v3.0 — InfoNCE Edition ===
✅ Loaded 20622 Advice samples


Epoch 1/20:   0%|          | 0/644 [00:00<?, ?it/s]/tmp/ipykernel_146102/3902516364.py:197: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  has_ctx = torch.tensor(has_ctx, dtype=torch.float32, device=DEVICE)
Epoch 1/20: 100%|██████████| 644/644 [01:05<00:00,  9.89it/s, anchor=0.9520, ctx=2.1182, hasctx=0.5373, loss=5.4571]


✅ Epoch 1 | Loss=5.4486 | Anchor=0.9505 | Context=2.1149 | HasCtx=0.5365


Epoch 2/20: 100%|██████████| 644/644 [01:02<00:00, 10.23it/s, anchor=0.0984, ctx=1.7833, hasctx=0.3746, loss=3.8522]


...

✅ Epoch 18 | Loss=3.5858 | Anchor=0.0475 | Context=1.7521 | HasCtx=0.0682


Epoch 19/20: 100%|██████████| 644/644 [01:02<00:00, 10.28it/s, anchor=0.0459, ctx=1.7495, hasctx=0.0655, loss=3.5776]


✅ Epoch 19 | Loss=3.5776 | Anchor=0.0459 | Context=1.7495 | HasCtx=0.0655


Epoch 20/20: 100%|██████████| 644/644 [01:03<00:00, 10.21it/s, anchor=0.0526, ctx=1.7556, hasctx=0.0628, loss=3.5953]


✅ Epoch 20 | Loss=3.5897 | Anchor=0.0525 | Context=1.7529 | HasCtx=0.0627

💾 Saved model & adapter to: models_anchor/dual_expert_Advice_v3_0


In [10]:
# ============================================================
# 🔍 Dual Expert v3.0 — TEST FILE (FIXED VERSION)
# ============================================================

import os
import torch
import torch.nn.functional as F
import pandas as pd
from sentence_transformers import SentenceTransformer


# ============================================================
# 🧠 Dual Expert Adapter (same as training)
# ============================================================

class DualExpertAdapter(torch.nn.Module):
    def __init__(self, input_dim=768, hidden_dim=256):
        super().__init__()

        self.anchor_adapter = torch.nn.Sequential(
            torch.nn.Linear(input_dim, hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, input_dim)
        )

        self.context_adapter = torch.nn.Sequential(
            torch.nn.Linear(input_dim, hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, input_dim)
        )

        self.has_context_head = torch.nn.Sequential(
            torch.nn.Linear(input_dim, hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, 1)
        )

    def forward(self, x):
        anchor_emb = F.normalize(self.anchor_adapter(x), p=2, dim=1)
        context_emb = F.normalize(self.context_adapter(x), p=2, dim=1)
        has_ctx_logit = self.has_context_head(x).squeeze(-1)
        return anchor_emb, context_emb, has_ctx_logit



# ============================================================
# ⚙ CONFIG
# ============================================================

MODEL_DIR = "models_anchor/dual_expert_Advice_v3_0"   # folder model bạn lưu
ADAPTER_PATH = f"{MODEL_DIR}/dual_adapter_v3.pt"
DATA_FILE = "Advice.csv"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TOP_K = 5



# ============================================================
# 🚀 LOAD MODEL
# ============================================================

print("🔄 Loading SentenceTransformer encoder...")
encoder = SentenceTransformer(MODEL_DIR).to(DEVICE)
encoder.eval()

try:
    emb_dim = encoder.get_sentence_embedding_dimension()
except:
    emb_dim = encoder.encode(["hi"], convert_to_tensor=True).shape[1]

print(f"📐 Embedding dimension = {emb_dim}")


print("🔄 Loading DualExpertAdapter...")
adapter = DualExpertAdapter(input_dim=emb_dim)
adapter.load_state_dict(torch.load(ADAPTER_PATH, map_location=DEVICE))
adapter = adapter.to(DEVICE)
adapter.eval()



# ============================================================
# 📥 LOAD ANCHORS + CONTEXTS
# ============================================================

df = pd.read_csv(DATA_FILE)
df = df[df["expert_label"] == "Advice"]

anchors = sorted(df["anchor"].unique())
contexts = sorted([c for c in df["context_name"].unique() if isinstance(c, str) and c.strip() != ""])

print(f"📌 Loaded {len(anchors)} anchors")
print(f"📌 Loaded {len(contexts)} contexts")



# ============================================================
# ⚡ PRE-ENCODE ANCHORS + CONTEXTS (FIXED)
# ============================================================

print("\n⚡ Encoding anchors...")
anchor_emb = encoder.encode(anchors, convert_to_tensor=True, device=DEVICE)
anchor_emb = anchor_emb.detach().clone()
anchor_emb = adapter.anchor_adapter(anchor_emb)
anchor_emb = F.normalize(anchor_emb, p=2, dim=1)


print("⚡ Encoding contexts...")
if len(contexts) > 0:
    context_emb = encoder.encode(contexts, convert_to_tensor=True, device=DEVICE)
    context_emb = context_emb.detach().clone()
    context_emb = adapter.context_adapter(context_emb)
    context_emb = F.normalize(context_emb, p=2, dim=1)
else:
    context_emb = None



# ============================================================
# 🔍 PREDICT FUNCTION
# ============================================================

def predict(question, top_k=TOP_K):
    print("\n==========================")
    print(f"🔎 QUESTION: {question}")
    print("==========================")

    # Encode question
    q_emb = encoder.encode([question], convert_to_tensor=True, device=DEVICE)
    q_emb = q_emb.detach().clone()

    q_anchor, q_ctx, q_hasctx = adapter(q_emb)

    # ---- ANCHOR SIM ----
    sim_anchor = (q_anchor @ anchor_emb.T)[0]
    top_anchor_idx = torch.topk(sim_anchor, top_k).indices.tolist()
    top_anchors = [(anchors[i], float(sim_anchor[i])) for i in top_anchor_idx]

    # ---- HAS-CONTEXT ----
    has_ctx_prob = torch.sigmoid(q_hasctx).item()

    # ---- CONTEXT SIM ----
    if context_emb is not None:
        sim_ctx = (q_ctx @ context_emb.T)[0]
        top_ctx_idx = torch.topk(sim_ctx, top_k).indices.tolist()
        top_contexts = [(contexts[i], float(sim_ctx[i])) for i in top_ctx_idx]
    else:
        top_contexts = []

    # PRINT
    print("\n🎯 TOP ANCHORS:")
    for name, score in top_anchors:
        print(f"  • {name:<30} → {score:.4f}")

    print(f"\n🔮 Has-Context Probability: {has_ctx_prob:.4f}")

    print("\n🌍 TOP CONTEXTS:")
    for name, score in top_contexts:
        print(f"  • {name:<30} → {score:.4f}")

    return {
        "anchors": top_anchors,
        "contexts": top_contexts,
        "has_context": has_ctx_prob
    }


# ============================================================
# 🧪 INTERACTIVE CHAT MODE (ENTER LIÊN TỤC)
# ============================================================

print("\n💬 Ready! Nhập câu hỏi để test (gõ 'exit' để thoát).")

while True:
    q = input("\n❓ Câu hỏi: ").strip()
    if q.lower() == "exit":
        print("👋 Bye!")
        break
    if q == "":
        continue   # enter rỗng thì bỏ qua
    
    predict(q)


🔄 Loading SentenceTransformer encoder...


The tokenizer you are loading from 'models_anchor/dual_expert_Advice_v3_0' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


📐 Embedding dimension = 768
🔄 Loading DualExpertAdapter...
📌 Loaded 474 anchors
📌 Loaded 5 contexts

⚡ Encoding anchors...
⚡ Encoding contexts...

💬 Ready! Nhập câu hỏi để test (gõ 'exit' để thoát).

🔎 QUESTION: Tôi bị HIV trong thai kỳ thì nên làm gì?

🎯 TOP ANCHORS:
  • HIV                            → 0.7663
  • AIDS                           → 0.5159
  • Đau khi quan hệ tình dục       → 0.3026
  • Virus Zika                     → 0.2919
  • Viêm gan siêu vi B             → 0.2770

🔮 Has-Context Probability: 0.9866

🌍 TOP CONTEXTS:
  • Thai kỳ                        → 0.7867
  • Trước thai kỳ                  → -0.0606
  • Cuối thai kỳ                   → -0.0759
  • Quá trình sinh nở              → -0.3710
  • Chuyển dạ                      → -0.3936

🔎 QUESTION: Tôi bị hiv trong thai kỳ thì nên làm gì

🎯 TOP ANCHORS:
  • HIV                            → 0.7456
  • AIDS                           → 0.4521
  • Viêm gan siêu vi B             → 0.2904
  • Ung thư âm hộ                 

KeyboardInterrupt: Interrupted by user

In [ ]:
# ============================================================
# 📊 Dual Expert v3.0 — EVALUATION SCRIPT
# ============================================================

import os
import torch
import torch.nn.functional as F
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics import f1_score


# ============================================================
# 🧠 Load Adapter (same as training)
# ============================================================

class DualExpertAdapter(torch.nn.Module):
    def __init__(self, input_dim=768, hidden_dim=256):
        super().__init__()

        self.anchor_adapter = torch.nn.Sequential(
            torch.nn.Linear(input_dim, hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, input_dim)
        )

        self.context_adapter = torch.nn.Sequential(
            torch.nn.Linear(input_dim, hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, input_dim)
        )

        self.has_context_head = torch.nn.Sequential(
            torch.nn.Linear(input_dim, hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, 1)
        )

    def forward(self, x):
        anchor_emb = F.normalize(self.anchor_adapter(x), p=2, dim=1)
        context_emb = F.normalize(self.context_adapter(x), p=2, dim=1)
        has_ctx_logit = self.has_context_head(x).squeeze(-1)
        return anchor_emb, context_emb, has_ctx_logit


# ============================================================
# ⚙ CONFIG
# ============================================================

MODEL_DIR = "models_anchor/dual_expert_Advice_v3_0"
ADAPTER_PATH = f"{MODEL_DIR}/dual_adapter_v3.pt"
DATA_FILE = "Advice_test.csv"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TOP_K = [1, 3, 5]


# ============================================================
# 🚀 Load Encoder + Adapter
# ============================================================

print("🔄 Loading SentenceTransformer...")
encoder = SentenceTransformer(MODEL_DIR).to(DEVICE)
encoder.eval()

try:
    emb_dim = encoder.get_sentence_embedding_dimension()
except:
    emb_dim = encoder.encode(["hi"], convert_to_tensor=True).shape[1]

print(f"📐 Embedding Dimension = {emb_dim}")

print("🔄 Loading Adapter...")
adapter = DualExpertAdapter(input_dim=emb_dim)
adapter.load_state_dict(torch.load(ADAPTER_PATH, map_location=DEVICE))
adapter = adapter.to(DEVICE)
adapter.eval()


# ============================================================
# 📥 Load Dataset Anchors + Contexts
# ============================================================

df = pd.read_csv(DATA_FILE)
df = df[df["expert_label"] == "Advice"]

anchors = sorted(df["anchor"].unique())
contexts = sorted([c for c in df["context_name"].unique() if isinstance(c, str) and c.strip() != ""])

print(f"📌 Found {len(anchors)} anchors")
print(f"📌 Found {len(contexts)} contexts")


# ============================================================
# ⚡ Pre-encode Anchors + Contexts
# ============================================================

print("⚡ Encoding anchors...")
anchor_emb = encoder.encode(anchors, convert_to_tensor=True, device=DEVICE)
anchor_emb = anchor_emb.detach().clone()
anchor_emb = adapter.anchor_adapter(anchor_emb)
anchor_emb = F.normalize(anchor_emb, p=2, dim=1)

print("⚡ Encoding contexts...")
if len(contexts) > 0:
    context_emb = encoder.encode(contexts, convert_to_tensor=True, device=DEVICE)
    context_emb = context_emb.detach().clone()
    context_emb = adapter.context_adapter(context_emb)
    context_emb = F.normalize(context_emb, p=2, dim=1)
else:
    context_emb = None


# ============================================================
# 📊 Evaluation Functions
# ============================================================

def topk_check(name_list, sim_scores, gold_name, k):
    top_k_idx = torch.topk(sim_scores, k).indices.tolist()
    top_k_names = [name_list[i] for i in top_k_idx]
    return gold_name in top_k_names


# ============================================================
# 📊 Main Evaluation
# ============================================================

def evaluate():
    print("\n🚀 Starting Evaluation...\n")

    anchor_correct = {1: 0, 3: 0, 5: 0}
    context_correct = {1: 0, 3: 0, 5: 0}
    hasctx_true = []
    hasctx_pred = []

    total = len(df)
    ctx_total = len(df[df["has_context"] == 1])

    for idx, row in df.iterrows():
        q = row["question"]
        gold_anchor = row["anchor"]
        gold_ctx = row["context_name"] if row["has_context"] == 1 else None
        gold_has_ctx = int(row["has_context"])

        # Encode question
        q_emb = encoder.encode([q], convert_to_tensor=True, device=DEVICE)
        q_emb = q_emb.detach().clone()

        q_anchor, q_ctx, q_hasctx = adapter(q_emb)

        # ---------- ANCHOR CHECK ----------
        sim_anchor = (q_anchor @ anchor_emb.T)[0]

        for k in TOP_K:
            if topk_check(anchors, sim_anchor, gold_anchor, k):
                anchor_correct[k] += 1

        # ---------- CONTEXT CHECK ----------
        if gold_ctx and context_emb is not None:
            sim_ctx = (q_ctx @ context_emb.T)[0]

            for k in TOP_K:
                if topk_check(contexts, sim_ctx, gold_ctx, k):
                    context_correct[k] += 1

        # ---------- HAS-CONTEXT ----------
        hasctx_pred_prob = torch.sigmoid(q_hasctx).item()
        hasctx_pred_bin = 1 if hasctx_pred_prob >= 0.5 else 0

        hasctx_true.append(gold_has_ctx)
        hasctx_pred.append(hasctx_pred_bin)

    # =======================================================
    # 📈 Final Metrics
    # =======================================================

    print("\n===============================")
    print("📊 FINAL EVALUATION RESULTS")
    print("===============================")

    print("\n🎯 Anchor Accuracy:")
    for k in TOP_K:
        print(f"  Top-{k}: {anchor_correct[k] / total:.4f}")

    print("\n🌍 Context Accuracy (only samples with context):")
    for k in TOP_K:
        print(f"  Top-{k}: {context_correct[k] / ctx_total:.4f}")

    print("\n🔮 Has-Context F1 Score:")
    print("  F1:", f"{f1_score(hasctx_true, hasctx_pred):.4f}")


if __name__ == "__main__":
    evaluate()


🔄 Loading SentenceTransformer...


The tokenizer you are loading from 'models_anchor/dual_expert_Advice_v3_0' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


📐 Embedding Dimension = 768
🔄 Loading Adapter...
📌 Found 137 anchors
📌 Found 5 contexts
⚡ Encoding anchors...
⚡ Encoding contexts...

🚀 Starting Evaluation...


📊 FINAL EVALUATION RESULTS

🎯 Anchor Accuracy:
  Top-1: 0.9948
  Top-3: 1.0000
  Top-5: 1.0000

🌍 Context Accuracy (only samples with context):
  Top-1: 1.0000
  Top-3: 1.0000
  Top-5: 1.0000

🔮 Has-Context F1 Score:
  F1: 0.9967


In [14]:
# ============================================================
# 🧪 Test Dual Expert v3.0 (Anchor + Context + HasContext)
# ============================================================

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from sentence_transformers import SentenceTransformer


# ------------------------------------------------------------
# ⚙️ CONFIGURATION
# ------------------------------------------------------------
EXPERT_NAME = "Advice"

MODEL_PATH = f"models_anchor/dual_expert_{EXPERT_NAME}_v3_0"
ADAPTER_PATH = os.path.join(MODEL_PATH, "dual_adapter_v3.pt")

ANCHOR_FILE = "anchors.csv"
CONTEXT_FILE = "contexts.csv"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

TOP_K = 5


# ------------------------------------------------------------
# 🧩 Load Anchors + Contexts
# ------------------------------------------------------------
df_anchor = pd.read_csv(ANCHOR_FILE)
ANCHORS = df_anchor["name"].dropna().tolist()

if os.path.exists(CONTEXT_FILE):
    CONTEXTS = pd.read_csv(CONTEXT_FILE)["name"].dropna().tolist()
else:
    CONTEXTS = []

print(f"✅ Loaded {len(ANCHORS)} anchors, {len(CONTEXTS)} contexts.")


# ------------------------------------------------------------
# 🧠 Dual Expert Adapter v3.0
# ------------------------------------------------------------
class DualExpertAdapter(nn.Module):
    def __init__(self, input_dim=768, hidden_dim=256):
        super().__init__()

        self.anchor_adapter = nn.Sequential(
            nn.Linear(input_dim, hidden_dim), nn.ReLU(), nn.Linear(hidden_dim, input_dim)
        )
        self.context_adapter = nn.Sequential(
            nn.Linear(input_dim, hidden_dim), nn.ReLU(), nn.Linear(hidden_dim, input_dim)
        )
        self.has_context_head = nn.Sequential(
            nn.Linear(input_dim, hidden_dim), nn.ReLU(), nn.Linear(hidden_dim, 1)
        )

    def forward(self, x):
        a = F.normalize(self.anchor_adapter(x), p=2, dim=1)
        c = F.normalize(self.context_adapter(x), p=2, dim=1)
        h = self.has_context_head(x).squeeze(-1)
        return a, c, h


# ------------------------------------------------------------
# 🚀 Load model + adapter
# ------------------------------------------------------------
print("\n🔄 Loading SentenceTransformer encoder...")
encoder = SentenceTransformer(MODEL_PATH).to(DEVICE)

try:
    EMB_DIM = encoder.get_sentence_embedding_dimension()
except:
    EMB_DIM = encoder.encode(["hi"], convert_to_tensor=True).shape[1]

adapter = DualExpertAdapter(input_dim=EMB_DIM).to(DEVICE)
adapter.load_state_dict(torch.load(ADAPTER_PATH, map_location=DEVICE))
adapter.eval()


# ------------------------------------------------------------
# 🧮 Helper — Encode function (v3.0)
# ------------------------------------------------------------
def encode_embed(texts):
    emb = encoder.encode(texts, convert_to_tensor=True, device=DEVICE)
    emb = emb.detach().clone()  # FIX inference mode
    return adapter(emb)          # anchor_emb, context_emb, hasctx


# ============================================================
# 🧪 INTERACTIVE TEST MODE
# ============================================================

print("\n💬 Ready! Nhập câu hỏi để test (gõ 'exit' để thoát).")

while True:
    QUESTION = input("\n❓ Câu hỏi: ").strip()
    if QUESTION.lower() == "exit":
        print("\n👋 Bye!")
        break
    if QUESTION == "":
        continue

    print(f"\n🧪 Testing expert '{EXPERT_NAME}' on question:\n➡️  \"{QUESTION}\"")

    # Encode câu hỏi
    q_a, q_c, q_h = encode_embed([QUESTION])

    # Encode anchors
    a_embs, _, _ = encode_embed(ANCHORS)

    # Encode contexts
    if len(CONTEXTS) > 0:
        _, c_embs, _ = encode_embed(CONTEXTS)
    else:
        c_embs = None

    # ----- Compute similarity -----
    sim_anchor = F.cosine_similarity(q_a, a_embs)
    sim_context = F.cosine_similarity(q_c, c_embs) if c_embs is not None else None
    has_ctx_prob = torch.sigmoid(q_h).item()

    # ----- Best anchor -----
    best_anchor = ANCHORS[torch.argmax(sim_anchor).item()]

    # ----- Best context -----
    if c_embs is not None and has_ctx_prob > 0.35:
        best_context = CONTEXTS[torch.argmax(sim_context).item()]
    else:
        best_context = "(no context)"

    # --------------------------------------------------------
    # 🔎 PRINT RESULTS
    # --------------------------------------------------------
    print("\n=== 🔎 RESULTS ===")
    print(f"🎯 Predicted Anchor : {best_anchor}")
    print(f"🌐 Predicted Context: {best_context}")
    print(f"📈 Has-Context prob : {has_ctx_prob:.4f}")

    print("\nTop 5 anchors:")
    for name, score in sorted(zip(ANCHORS, sim_anchor.tolist()), key=lambda x: x[1], reverse=True)[:TOP_K]:
        print(f"{name:<40} → {score:.4f}")

    if c_embs is not None:
        print("\nTop 5 contexts:")
        for name, score in sorted(zip(CONTEXTS, sim_context.tolist()), key=lambda x: x[1], reverse=True)[:TOP_K]:
            print(f"{name:<40} → {score:.4f}")


✅ Loaded 409 anchors, 5 contexts.

🔄 Loading SentenceTransformer encoder...


The tokenizer you are loading from 'models_anchor/dual_expert_Advice_v3_0' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.



💬 Ready! Nhập câu hỏi để test (gõ 'exit' để thoát).

🧪 Testing expert 'Advice' on question:
➡️  "nên làm gì khi bị cao huyết áp trong thai kỳ"

=== 🔎 RESULTS ===
🎯 Predicted Anchor : Cao huyết áp
🌐 Predicted Context: Thai kỳ
📈 Has-Context prob : 0.9959

Top 5 anchors:
Cao huyết áp                             → 0.6793
Cao huyết áp mãn tính                    → 0.4766
Tăng huyết áp                            → 0.4111
Huyết áp cao                             → 0.3806
Tăng huyết áp mãn tính                   → 0.3542

Top 5 contexts:
Thai kỳ                                  → 0.8825
Cuối thai kỳ                             → -0.0568
Trước khi thai kỳ                        → -0.3305
Quá trình sinh nở                        → -0.3572
Chuyển dạ                                → -0.3789


KeyboardInterrupt: Interrupted by user

In [ ]:
# ============================================================
# 🧪 Test Dual Expert v3.0 (with Anchor Layer Output)
# ============================================================

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from sentence_transformers import SentenceTransformer


# ------------------------------------------------------------
# ⚙️ CONFIGURATION
# ------------------------------------------------------------
EXPERT_NAME = "Advice"

MODEL_PATH = f"models_anchor/dual_expert_{EXPERT_NAME}_v3_0"
ADAPTER_PATH = os.path.join(MODEL_PATH, "dual_adapter_v3.pt")

ANCHOR_FILE = "anchors.csv"
CONTEXT_FILE = "contexts.csv"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

TOP_K = 5


# ------------------------------------------------------------
# 🧩 Load Anchors + Layers + Contexts
# ------------------------------------------------------------
df_anchor = pd.read_csv(ANCHOR_FILE)

ANCHORS = df_anchor["name"].tolist()
ANCHOR_LAYERS = df_anchor["layer"].tolist()   # ⬅️ LAYER HERE

if os.path.exists(CONTEXT_FILE):
    CONTEXTS = pd.read_csv(CONTEXT_FILE)["name"].dropna().tolist()
else:
    CONTEXTS = []

print(f"✅ Loaded {len(ANCHORS)} anchors, {len(CONTEXTS)} contexts.")


# ------------------------------------------------------------
# 🧠 Dual Expert Adapter v3.0
# ------------------------------------------------------------
class DualExpertAdapter(nn.Module):
    def __init__(self, input_dim=768, hidden_dim=256):
        super().__init__()

        self.anchor_adapter = nn.Sequential(
            nn.Linear(input_dim, hidden_dim), nn.ReLU(), nn.Linear(hidden_dim, input_dim)
        )
        self.context_adapter = nn.Sequential(
            nn.Linear(input_dim, hidden_dim), nn.ReLU(), nn.Linear(hidden_dim, input_dim)
        )
        self.has_context_head = nn.Sequential(
            nn.Linear(input_dim, hidden_dim), nn.ReLU(), nn.Linear(hidden_dim, 1)
        )

    def forward(self, x):
        a = F.normalize(self.anchor_adapter(x), p=2, dim=1)
        c = F.normalize(self.context_adapter(x), p=2, dim=1)
        h = self.has_context_head(x).squeeze(-1)
        return a, c, h


# ------------------------------------------------------------
# 🚀 Load model + adapter
# ------------------------------------------------------------
print("\n🔄 Loading SentenceTransformer encoder...")
encoder = SentenceTransformer(MODEL_PATH).to(DEVICE)

try:
    EMB_DIM = encoder.get_sentence_embedding_dimension()
except:
    EMB_DIM = encoder.encode(["hi"], convert_to_tensor=True).shape[1]

adapter = DualExpertAdapter(input_dim=EMB_DIM).to(DEVICE)
adapter.load_state_dict(torch.load(ADAPTER_PATH, map_location=DEVICE))
adapter.eval()


# ------------------------------------------------------------
# 🧮 Helper — Encode function (v3.0)
# ------------------------------------------------------------
def encode_embed(texts):
    emb = encoder.encode(texts, convert_to_tensor=True, device=DEVICE)
    emb = emb.detach().clone()  # FIX inference mode
    return adapter(emb)          # anchor_emb, context_emb, hasctx


# ============================================================
# 🧪 INTERACTIVE TEST MODE
# ============================================================

print("\n💬 Ready! Nhập câu hỏi để test (gõ 'exit' để thoát).")

while True:
    QUESTION = input("\n❓ Câu hỏi: ").strip()
    if QUESTION.lower() == "exit":
        print("\n👋 Bye!")
        break
    if QUESTION == "":
        continue

    print(f"\n🧪 Testing expert '{EXPERT_NAME}' on question:\n➡️  \"{QUESTION}\"")

    # Encode câu hỏi
    q_a, q_c, q_h = encode_embed([QUESTION])

    # Encode anchors & contexts
    a_embs, _, _ = encode_embed(ANCHORS)
    if len(CONTEXTS) > 0:
        _, c_embs, _ = encode_embed(CONTEXTS)
    else:
        c_embs = None

    # ----- Similarity -----
    sim_anchor = F.cosine_similarity(q_a, a_embs)
    sim_context = F.cosine_similarity(q_c, c_embs) if c_embs is not None else None
    has_ctx_prob = torch.sigmoid(q_h).item()

    # ----- Best anchor -----
    best_idx = torch.argmax(sim_anchor).item()
    best_anchor = ANCHORS[best_idx]
    best_layer = ANCHOR_LAYERS[best_idx]  # ⬅️ PRINT LAYER

    # ----- Best context -----
    if c_embs is not None and has_ctx_prob > 0.35:
        best_context = CONTEXTS[torch.argmax(sim_context).item()]
    else:
        best_context = "(no context)"

    # --------------------------------------------------------
    # 🔎 PRINT RESULTS (now with layer!)
    # --------------------------------------------------------
    print("\n=== 🔎 RESULTS ===")
    print(f"🎯 Predicted Anchor : {best_anchor}   ({best_layer})")
    print(f"🌐 Predicted Context: {best_context}")
    print(f"📈 Has-Context prob : {has_ctx_prob:.4f}")

    print("\nTop 5 anchors:")
    top5 = sorted(
        list(zip(ANCHORS, ANCHOR_LAYERS, sim_anchor.tolist())),
        key=lambda x: x[2], reverse=True
    )[:TOP_K]

    for name, layer, score in top5:
        print(f"{name:<35} ({layer:<12}) → {score:.4f}")

    if c_embs is not None:
        print("\nTop 5 contexts:")
        top5_ctx = sorted(
            list(zip(CONTEXTS, sim_context.tolist())),
            key=lambda x: x[1], reverse=True
        )[:TOP_K]

        for name, score in top5_ctx:
            print(f"{name:<40} → {score:.4f}")


✅ Loaded 408 anchors, 5 contexts.

🔄 Loading SentenceTransformer encoder...


The tokenizer you are loading from 'models_anchor/dual_expert_Advice_v3_0' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.



💬 Ready! Nhập câu hỏi để test (gõ 'exit' để thoát).

🧪 Testing expert 'Advice' on question:
➡️  "Tôi bị Tiền tiểu đường thì nên làm gì"

=== 🔎 RESULTS ===
🎯 Predicted Anchor : Tiền tiểu đường   (Disease)
🌐 Predicted Context: (no context)
📈 Has-Context prob : 0.0004

Top 5 anchors:
Tiền tiểu đường                     (Disease     ) → 0.7076
Tiểu đường                          (Disease     ) → 0.4100
Tiền sản giật                       (Disease     ) → 0.3636
Tiền sản giật                       (Topic       ) → 0.3636
Chẩn đoán tiền sản                  (Topic       ) → 0.2764

Top 5 contexts:
Thai kỳ                                  → 0.3170
Cuối thai kỳ                             → 0.2872
Chuyển dạ                                → 0.1193
Quá trình sinh nở                        → -0.2940
Trước khi thai kỳ                        → -0.4054
